# Project on Full Dataset

In [1]:
import os
import numpy as np
import scipy as sp
import scipy.stats as sts
import matplotlib.pyplot as plt
import time

from collections import defaultdict
from glob import glob
import random as rd
import pylab as pl
import pandas as pd
import re
from sklearn import decomposition
from sklearn import pipeline


from PIL import Image
import PIL.ImageOps

#from skimage import io, color
import matplotlib.image as mpimg
from skimage.transform import resize

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, cross_val_score,  GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import classification_report, mean_squared_error

import seaborn as sns
from sklearn.metrics import confusion_matrix

import pygeos
import ntpath
import geopandas as gpd
#import geoplot
from shapely.geometry import Point, Polygon
from fiona.crs import from_epsg
import math

import tensorflow as tf
import gc

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-9k7v2it7 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_HOME'] = '/workspace/cache'

In [3]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES= 0,1,2,3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0,1,2,3


In [4]:
# set the random seed for reproducibility
rd.seed(3)

In [5]:
tf.config.list_logical_devices('GPU')

[LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU'),
 LogicalDevice(name='/device:GPU:2', device_type='GPU'),
 LogicalDevice(name='/device:GPU:3', device_type='GPU')]

## Import Data

In [6]:
# Function to resize the data and flatten images if necessary
def transform(dataset):
    new_list = []
    for i in range(len(dataset)):
        #resize the image
        temp2 = resize(dataset[i], (256, 256), Image.NEAREST)
        
        #flatten it
        new_list.append(temp2.flatten())

    return new_list

In [7]:
def imagePreProcess(pathLink, outcomeLink, n_samples = 100, n_comps_filter = 30):
    """
    func: Filteres and preprocesses the images for one country
    
    input:
        -pathLink: path to image files
        -outcomeLink: path to outcome file
        -n_samples: no. of samples in PCA, default 100
        -n_comps_filter: no. of components to filter, default 30 
        
    
    output: 
        -indices for filtered array
        -transformed images
        -geoframe with matched images
    """
    
    print("Opening files from", pathLink)
    
    #get all the pathlinks to the images
    raw_images_links = glob(pathLink)
    
    #open the images
    img_arrays = [np.array(Image.open(file_name)) for file_name in raw_images_links]
    
    #transform images
    img_transform = transform(img_arrays)
    
    print("Filtering Images")
    
    #filter the images using a PCA
    n_samples = 100 #set higher for more accuracy 
    n_comps_filter = 30 #set higher for more accuracy 
    
    pca_filter = PCA(n_comps_filter)
    pca_filter.fit(rd.sample(img_transform, n_samples))
    print(f"Completed training the PCA for filtering with {n_samples} samples and {n_comps_filter} components. \nvar expl: {round(np.cumsum(pca_filter.explained_variance_ratio_)[n_comps_filter-1]*100)}%")
    img_pca_filter = pca_filter.transform(img_transform)
    
    #filter out the unique images
    img_unique, unique_indices = np.unique(img_pca_filter, axis= 0, return_index= True)
    
    print("Finding Location of images")
    # Get the location of the images 
    image_geodf = gpd.GeoDataFrame() # Create an empty geopandas GeoDataFrame
    for i in unique_indices:
        #extract an array of the geo points from the file name 
        point_array = [float(point) for point in str.split(ntpath.basename(raw_images_links[i][:-4]),"_")]

        lat_i = 1
        long_i = 0
        #calculate the width of the image taken
        #https://wiki.openstreetmap.org/wiki/Zoom_levels 
        meters = 40075016.686 * math.cos(math.radians(point_array[lat_i]))/(2**14)

        #the tiles are x-meters wide. Let's find that in degrees: 
        # (https://stackoverflow.com/questions/25237356/convert-meters-to-decimal-degrees)
        width_deg_half = (meters / (111.32 * 1000 * math.cos(point_array[lat_i] * (math.pi / 180))))/2

        geo_point = [(point_array[lat_i] - width_deg_half, point_array[long_i]+ width_deg_half),
                    (point_array[lat_i] + width_deg_half, point_array[long_i]+ width_deg_half),
                    (point_array[lat_i] + width_deg_half, point_array[long_i] - width_deg_half),
                    (point_array[lat_i] - width_deg_half, point_array[long_i] - width_deg_half)]

        # Create a Shapely polygon from the coordinate-tuple list
        image_geodf.loc[i, 'geometry'] = Polygon(geo_point)
        # assign an image id. used for merging with actual images and the survey values
        image_geodf.loc[i, 'image_id'] = i  

        # Set the GeoDataFrame's coordinate system to WGS84 (i.e. epsg code 4326)
    image_geodf.crs = from_epsg(4326)
    
    print("Loading Outcome")
    
    #load outcome data
    outcome_geodf = gpd.read_file(outcomeLink)
    
    
    geo_df = gpd.sjoin(image_geodf, outcome_geodf, how="inner", predicate = "contains")
    
    ## ASSEMBLE DATASET FOR COUNTRY
    X_country = []
    Y_country = []
    for i in range(len(geo_df)):
        obs = geo_df.iloc[i]
        X_country.append(img_transform[int(obs['image_id'])])
        Y_country.append(obs['wealth'])
    
    return X_country, Y_country




In [ ]:
X_mala, Y_mala =  imagePreProcess("image_data/malawi_archive/images/*", 
                                  "outcome_data/mlw_gdf.shp")

X_mali, Y_mali = imagePreProcess("image_data/Mali_archive/images/*", 
                                "outcome_data/mli_gdf.shp")

X_ni, Y_ni = imagePreProcess("image_data/nigeria_archive/images/*", 
                                "outcome_data/ngr_gdf.shp")

X_eth, Y_eth = imagePreProcess("image_data/ethiopia_archive/images/ethiopia_Images/*", 
                                "outcome_data/eth_gdf.shp")

Opening files from image_data/malawi_archive/images/*
Filtering Images
Completed training the PCA for filtering with 100 samples and 30 components. 
var expl: 89%


In [ ]:
#gpd.read("outcome_data/mlw_gdf.shp")

In [ ]:
print(len(X_mala))
print(len(X_mali))
print(len(X_ni))
print(len(X_eth))

In [ ]:
# List of X dataframes
X_full = X_mala + X_mali + X_eth + X_ni

# List of Y dataframes
Y_full = Y_mala + Y_mali + Y_eth + Y_ni

print(len(X_full))

### Train Test Split

In [ ]:
#split into train and test data set
X_train, X_test, y_train, y_test = train_test_split(X_full, Y_full, test_size=0.15)

## PCA

In [ ]:
# Let's find how many components we should use by just applying a PCA with a range of components

#initialize the pca
pca = PCA(whiten=True, copy=True, random_state= 10)
#fit it
pca.fit(rd.sample(X_train,100))

#get the cumulative 
total = np.cumsum(pca.explained_variance_ratio_)
total[26]

In [ ]:
plt.figure(figsize=(15, 10))
comp_to_view = 25
#let's plot it
plot_xs = list(range(comp_to_view))


plt.scatter(plot_xs, pca.explained_variance_ratio_[:comp_to_view], color='black', label='Explained variance proportion')
plt.scatter(plot_xs, total[:comp_to_view],color='orange', label='Total')

plt.legend(loc=2)
plt.xlabel('Number of Component', size=16)
plt.ylabel('Variance Explained (%)', size=16)
#plt.hlines(0.,xmin=0, xmax=comp_to_view, colors = 'blue', linestyles='--')
plt.grid('minor')
plt.show()

print(pca.explained_variance_ratio_[:comp_to_view])

In [ ]:
#initialize the pca

n_comp = 25
pca = PCA(n_components = n_comp, whiten=True, copy=True, random_state= 10)

#fit with random sample of _ observations
pca.fit(rd.sample(X_train,10000))


In [ ]:
## batchwise transformation of the 

reduced = []
batchsize = 2500
# 
for i in range(0, len(X_train), batchsize):
    if i == 0:
        reduced = pca.transform(X_train[i:i+batchsize])
        continue 
    
    reduced_batch = pca.transform(X_train[i:i+batchsize])
    reduced = np.concatenate((reduced, reduced_batch), axis =0)

X_train_pca = np.asarray(reduced)
n_obs = len(X_train_pca)

In [ ]:
import matplotlib.image as mpimg
#function straight copied from
def plot_gallery(title, images, n_col, n_row, cmap_p=plt.cm.gray):
    plt.figure(figsize=(2. * n_col, 2.26 * n_row))
    plt.suptitle(title, size=16)
    for i, comp in enumerate(images):
        plt.subplot(n_row, n_col, i + 1)
        vmax = max(comp.max(), -comp.min())
        #print(vmax)
        plt.imshow((comp*255).astype(np.uint8),
                   interpolation='nearest',
                   vmin=-vmax, vmax=vmax 
                   #cmap = cmap_p
                   )
        plt.xticks(())
        plt.yticks(())
    plt.subplots_adjust(0.01, 0.05, 0.99, 0.93, 0.04, 0.)
    return plt.show()

In [ ]:
eigenimages = pca.components_.reshape(n_comp,256,256,4).copy()
plot_gallery("PCA", eigenimages[:,:,:,:3], n_col=5, n_row=5, cmap_p = plt.cm.viridis)

## Linear Model

In [ ]:
lin_model = LinearRegression()
lin_model.fit(X_train_pca, y_train)

In [ ]:
lin_model.score(X_train_pca, y_train)

In [ ]:
len(np.unique(X_train_pca))

In [ ]:
plt.hist((y_train))
plt.show()

## Random Forest

In [ ]:
rf = RandomForestRegressor(n_estimators= 100, bootstrap = True)
rf.fit(X_train_pca, y_train)

In [ ]:
rf.score(X_train_pca, y_train)

## Running Grid Search on Random Forest Regressor

Source and documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

Tuning: 
- n_estimators: number of trees in the forest
- max_depth: maximum depth of the tree 
- max_features: number of features taken into account when splitting. 

In [ ]:
hyper_n_list = [100]
hyper_depth_list = [1,2,3,4,5, 10,16,18,20,22,24,26, 30, 40, 50, 100, 200]
hyper_m_list = ["auto", "sqrt", "log2"]

In [ ]:
grid_search = GridSearchCV(RandomForestRegressor(random_state=0),
                           {
                              'n_estimators':hyper_n_list,
                              'max_depth': hyper_depth_list,
                              'max_features': hyper_m_list,
                            },cv=4, scoring="r2",verbose=1,n_jobs=-1
                           )
grid_search.fit(X_train_pca,y_train)

In [ ]:
cv_df = pd.DataFrame(grid_search.cv_results_)

In [ ]:
cv_df

In [ ]:
def hyperparam_plot2d(df, param, outcome, group_var = None):

    '''
    func: function to plot the performance of a parameter
    - df (dataframe): cross-validation grid search dataframe

    params have to be column names in DF
    - param (str): which parameter to asses
    - outcome (str): which outcome to find the max for
    - group_var (str): grouping variable (default None)
    '''

    group_var_list = np.unique(np.array(df[group_var]))
    plt.figure(figsize=(12,8))
    #print(np.argmax(np.asarray(df[outcome])))
    best_score = np.max(np.asarray(df[outcome]))
    best_score_param = np.asarray(df[param])[np.argmax(np.asarray(df[outcome]))]
    if group_var != None:
        best_score_group = np.asarray(df[group_var])[np.argmax(np.asarray(df[outcome]))]
        #print(best_score_group)

    if group_var == None:
        plt.plot(param,outcome,data = df)
        plt.xlabel(param)
        plt.ylabel(outcome)
        #plt.grid()
        plt.vlines(best_score_param, 0, np.max(np.asarray(df[outcome])+.05))
        plt.text(best_score_param,best_score - (best_score*.1),  f'Best parameter: {best_score_param} with score: {best_score}',size =12)
        plt.legend()
    else:
        for inst in group_var_list:
            plt.plot(param,outcome,label = inst, data = df[df[group_var]== inst])
            plt.xlabel(param)
            plt.ylabel(outcome)
            #plt.grid()
            plt.text(best_score_param,best_score - (best_score*.1),  f'Best parameter: {best_score_param} \n with score: {round(best_score,3)} \nin group:{best_score_group}', size = 12)
            plt.vlines(best_score_param, 0, best_score+.05)
            plt.legend(loc = 4)
            plt.ylim(bottom =0 )
    plt.grid()
    return plt.show()

In [ ]:
hyperparam_plot2d(cv_df, 'param_max_depth', 'mean_test_score', 'param_max_features')

In [ ]:
hyperparam_plot2d(cv_df, 'param_max_features' ,'mean_test_score','param_max_depth')

In [ ]:
best_forest = grid_search.best_estimator_

## CNN 

### Prepare

In [22]:
from tensorflow import keras
import gc
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras import applications

In [ ]:
X_train_cnn = np.asarray(X_train).reshape(n_obs, 256, 256, 4)
#X_train_cnn

In [ ]:
X_train_cnn = X_train_cnn[:,:,:,:3].astype('float32') # Taking out the fourth channel as ghe information is the same everywhere
y_train_cnn = np.asarray(y_train).astype('float32')

In [ ]:
X_train1_cnn, X_val_cnn, y_train1_cnn, y_val_cnn = train_test_split(X_train_cnn, y_train_cnn, test_size=0.10)

### Model 

In [24]:
# https://jmlb.github.io/ml/2017/03/20/CoeffDetermination_CustomMetric4Keras/
from keras import backend as K

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
cnn_satellite = Sequential()
cnn_satellite.add(Conv2D(32, kernel_size=(16, 16),activation='linear',input_shape=(256,256,3),padding='same'))
cnn_satellite.add(LeakyReLU(alpha=0.1))
cnn_satellite.add(MaxPooling2D((2, 2),padding='same'))
cnn_satellite.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
cnn_satellite.add(LeakyReLU(alpha=0.1))
cnn_satellite.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
cnn_satellite.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
cnn_satellite.add(LeakyReLU(alpha=0.1))                  
cnn_satellite.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
cnn_satellite.add(Flatten())
cnn_satellite.add(Dense(32, activation='linear'))
cnn_satellite.add(LeakyReLU(alpha=0.1))                  
cnn_satellite.add(Dense(1, activation='linear'))

gc.collect()

In [ ]:
cnn_satellite.compile(loss=keras.losses.mean_squared_error, 
                      optimizer=keras.optimizers.Adam(),
                      metrics=[coeff_determination])

In [ ]:
cnn_satellite.summary()

In [ ]:
#os.environ['CUDA_LAUNCH_BLOCKING'] = "0"
#os.environ['TORCH_HOME'] = '/workspace/cache'

In [12]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [20]:
gc.collect()

954

In [25]:
with strategy.scope():
    cnn_satellite = Sequential()
    cnn_satellite.add(Conv2D(32, kernel_size=(16, 16),activation='linear',input_shape=(256,256,3),padding='same'))
    cnn_satellite.add(LeakyReLU(alpha=0.1))
    cnn_satellite.add(MaxPooling2D((2, 2),padding='same'))
    cnn_satellite.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
    cnn_satellite.add(LeakyReLU(alpha=0.1))
    cnn_satellite.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    cnn_satellite.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
    cnn_satellite.add(LeakyReLU(alpha=0.1))                  
    cnn_satellite.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    cnn_satellite.add(Flatten())
    cnn_satellite.add(Dense(32, activation='linear'))
    cnn_satellite.add(LeakyReLU(alpha=0.1))                  
    cnn_satellite.add(Dense(1, activation='linear'))
    cnn_satellite.compile(loss=keras.losses.mean_squared_error, 
                      optimizer=keras.optimizers.Adam(),
                      metrics=[coeff_determination])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [ ]:
X_cnn_res50 = applications.resnet_v2.preprocess_input(
    X_train1_cnn)

In [ ]:
## Hyperparameters

epochs = 2
batchsize = 5

In [ ]:
#fit and record the history 
cnn_history = cnn_satellite.fit(X_cnn_res50, 
                  y_train1_cnn, 
                  batch_size=batchsize, 
                  epochs=epochs, 
                  verbose=1
                  #validation_data=(X_val_cnn, y_val_cnn)
                                   )

In [ ]:
test_eval = cnn_satellite.evaluate(X_val_cnn, y_val_cnn, verbose=1)

In [ ]:
plt.plot([i for i in range(epochs)], cnn_history.history['val_coeff_determination'], color = 'grey')
plt.scatter([i for i in range(epochs)], cnn_history.history['val_coeff_determination'])
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Validation $R^2$')
plt.show()